In [356]:
import pandas as pd
import seaborn as sns
import numpy as np
import collections
import matplotlib.pyplot as plt

In [509]:
name_recodes = {
    "USA (EOIR)":"United States",
    "USA (INS/DHS)":"United States",
    "Venezuela, Bolivarian Republic of":"Venezuela",
    "Venezuela (Bolivarian Republic of)":"Venezuela"
}

country_codes = pd.read_csv("country_codes.csv")
country_codes['Name'] = country_codes['Name'].replace(name_recodes)

In [510]:
####### UNHCR

unhcr_raw = pd.read_csv("unhcr_popstats_export_asylum_seekers_monthly_all_data.csv", skiprows=3)

# replacing names
unhcr_raw['Origin'] = unhcr_raw['Origin'].replace(name_recodes)
unhcr_raw['Country / territory of asylum/residence'] = unhcr_raw['Country / territory of asylum/residence'].replace(name_recodes)

unhcr_temp_1 = pd.merge(unhcr_raw, country_codes, left_on="Country / territory of asylum/residence", right_on="Name", how="left")
unhcr_temp_2 = pd.merge(unhcr_temp_1, country_codes, left_on='Origin', right_on='Name', suffixes=('_destination','_origin'), how="left")
unhcr_temp_3 = unhcr_temp_2[['Code_destination','Code_origin','Year','Month','Value','Name_destination','Name_origin']].drop_duplicates()

# since 2014
unhcr = unhcr_temp_3[(unhcr_temp_3['Year'] >= 2014) & (unhcr_temp_3['Year'] <= 2017)]

# weekly
unhcr['date_recode'] = pd.to_datetime(unhcr['Month'] + ' ' + unhcr['Year'].astype(str))
unhcr['Value'] = pd.to_numeric(unhcr['Value'], errors='coerce')
unhcr = unhcr.groupby(['Name_origin','Name_destination','Code_destination','Code_origin','date_recode']).aggregate({"Value":"sum"}).reset_index()


# countries after 2014 to csv
unhcr_temp_2_out = unhcr_temp_2[unhcr_temp_2['Year'] >= 2014][['Origin','Code_origin']].drop_duplicates()
unhcr_temp_2_out.to_csv('origin_codes.csv', index=False)



/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [511]:
####### GTRENDS PREP
kw_list = ["inmigrar+EEUU","inmigrar+españa","asilo+EEUU","asilo+españa","emigrar+EEUU","emigrar+españa"] # this needs to come from gtrends.py
renames_ = {str(kw):kw.replace("+","_") for kw in kw_list} # can't have +s in variable names
new_kw_list = list(renames_.values())
aggregation = {kw:"mean" for kw in new_kw_list}

# insheet google trends
gtrends_raw = pd.read_csv("gtrends_.csv")
gtrends = gtrends_raw.rename(columns=renames_)
gtrends['date_recode'] = pd.to_datetime(pd.to_datetime(gtrends['index']).dt.strftime('%Y-%m-01'))
gtrends_monthly = gtrends.groupby(['Country Code','date_recode']).aggregate(aggregation).reset_index()

In [512]:
'''for current_country in list_of_countries:
    combined_current_country = combined[combined['Country Code'] == current_country]
    combined_current_country['norm_tone'] = (combined_current_country['Tone'] - np.mean(combined_current_country['Tone']))/np.std(combined_current_country['Tone'])
    combined_current_country['norm_refugees'] = (combined_current_country['refugees'] - np.mean(combined_current_country['refugees']))/np.std(combined_current_country['refugees'])
    combined_current_country = combined_current_country.set_index('date_recode')
    as_twoseries = [combined_current_country['norm_tone'],combined_current_country['norm_refugees']]
    plt.figure(figsize=(10,10))
    ax = sns.lineplot(data=as_twoseries, sort=True).set_title(current_country)'''

"for current_country in list_of_countries:\n    combined_current_country = combined[combined['Country Code'] == current_country]\n    combined_current_country['norm_tone'] = (combined_current_country['Tone'] - np.mean(combined_current_country['Tone']))/np.std(combined_current_country['Tone'])\n    combined_current_country['norm_refugees'] = (combined_current_country['refugees'] - np.mean(combined_current_country['refugees']))/np.std(combined_current_country['refugees'])\n    combined_current_country = combined_current_country.set_index('date_recode')\n    as_twoseries = [combined_current_country['norm_tone'],combined_current_country['norm_refugees']]\n    plt.figure(figsize=(10,10))\n    ax = sns.lineplot(data=as_twoseries, sort=True).set_title(current_country)"

In [513]:
##### GTRENDS + UNHCR

refugees_raw = pd.merge(gtrends_monthly, unhcr, left_on=['date_recode','Country Code'], right_on=['date_recode','Code_origin'])
refugees = refugees_raw[((refugees_raw['Code_destination'] == "ES") | (refugees_raw['Code_destination'] == "US"))][['Value','date_recode','Code_origin','Code_destination','Name_origin','Name_destination']+new_kw_list]
refugees['Value'] = pd.to_numeric(refugees['Value'], errors='coerce')

# fill NAs with 0 in keyword columns
refugees = refugees.fillna(0)


In [514]:
refugees = refugees.drop_duplicates().sort_values(['Code_origin','Code_destination','date_recode']).dropna(subset=['Value'])
refugees.to_csv("refugees.csv", index=False)


In [515]:
# first keep only pairs where we have full years of data
refugees['year'] = refugees['date_recode'].dt.year
refugees['num_in_year'] = refugees.groupby(['Code_origin','Code_destination','year'])['date_recode'].transform('count')
refugees_fullyears = refugees[refugees.num_in_year == 12]
refugees_fullyears = refugees_fullyears.drop_duplicates().sort_values(['Code_origin','Code_destination','date_recode']).dropna(subset=['Value'])

refugees_wlags = refugees_fullyears
#refugees_wlags['Value_lag1'] = refugees_wlags.groupby(['Code_origin','Code_destination']).Value.rolling(window=1, win_type='boxcar', center=False).sum().reset_index()['num_migrants'].shift(-4)
refugees_wlags['Value_lag1'] = refugees_wlags.groupby(['Code_origin','Code_destination']).Value.shift(1)
refugees_wlags['Value_lag2'] = refugees_wlags.groupby(['Code_origin','Code_destination']).Value.shift(2)
refugees_wlags['Value_diff'] = refugees_wlags['Value'] - refugees_wlags['Value_lag1']


for kw in new_kw_list:
    refugees_wlags[kw+'_lag1'] = refugees_wlags.groupby(['Code_origin','Code_destination'])[kw].shift(1)
    refugees_wlags[kw+'_lag2'] = refugees_wlags.groupby(['Code_origin','Code_destination'])[kw].shift(2)
    refugees_wlags[kw+'_diff1'] = refugees_wlags[kw] - refugees_wlags[kw+'_lag1']

# differences here: we want to predict next month's change in asylum applications by searches this month

    
refugees_wlags.to_csv('refugees_wlags.csv', index=False)
    

In [516]:
refugees_wlags

,Value,date_recode,Code_origin,Code_destination,Name_origin,Name_destination,inmigrar_EEUU,inmigrar_españa,asilo_EEUU,asilo_españa,...,asilo_EEUU_diff1,asilo_españa_lag1,asilo_españa_lag2,asilo_españa_diff1,emigrar_EEUU_lag1,emigrar_EEUU_lag2,emigrar_EEUU_diff1,emigrar_españa_lag1,emigrar_españa_lag2,emigrar_españa_diff1
107,0.0,2017-01-01,CL,US,Chile,United States,4.20,8.40,4.80,8.80,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
110,0.0,2017-02-01,CL,US,Chile,United States,3.50,9.25,4.00,9.25,...,-0.80,8.80,NaN,0.45,4.80,NaN,-1.05,8.20,NaN,1.05
114,5.0,2017-03-01,CL,US,Chile,United States,3.75,8.25,3.75,9.00,...,-0.25,9.25,8.80,-0.25,3.75,4.80,0.25,9.25,8.20,-0.25
117,0.0,2017-04-01,CL,US,Chile,United States,10.20,8.80,10.20,9.20,...,6.45,9.00,9.25,0.20,4.00,3.75,6.40,9.00,9.25,0.20
120,0.0,2017-05-01,CL,US,Chile,United States,4.75,9.00,4.50,9.25,...,-5.70,9.20,9.00,0.05,10.40,4.00,-5.90,9.20,9.00,-0.20
124,6.0,2017-06-01,CL,US,Chile,United States,4.50,9.75,4.75,10.00,...,0.25,9.25,9.20,0.75,4.50,10.40,0.25,9.00,9.20,1.00
128,0.0,2017-07-01,CL,US,Chile,United States,4.20,8.80,4.60,9.00,...,-0.15,10.00,9.25,-1.00,4.75,4.50,-0.35,10.00,9.00,-1.40
132,0.0,2017-08-01,CL,US,Chile,United States,5.25,11.50,5.25,11.75,...,0.65,9.00,10.00,2.75,4.40,4.75,1.10,8.60,10.00,2.90
136,0.0,2017-09-01,CL,US,Chile,United States,6.25,9.00,6.75,9.50,...,1.50,11.75,9.00,-2.25,5.50,4.40,0.50,11.50,8.60,-2.00
143,0.0,2017-10-01,CL,US,Chile,United States,4.20,10.00,4.20,10.80,...,-2.55,9.50,11.75,1.30,6.00,5.50,-1.60,9.50,11.50,0.90
